In [5]:
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark import SparkContext


spark = SparkSession.builder.appName('abc').getOrCreate()
sc = spark.sparkContext


tmp_data = [(100,"aaa"),(120,"bbb"),(150,"ccc"),(200,"aaa"),(220,"aaa")]
tmp_row = Row("id","name")
rdd_row = sc.parallelize(tmp_data)
rdd_schema = rdd_row.map(lambda x: tmp_row(*x))


df_employee = spark.createDataFrame(rdd_schema)
df_employee.show()

+---+----+
| id|name|
+---+----+
|100| aaa|
|120| bbb|
|150| ccc|
|200| aaa|
|220| aaa|
+---+----+



In [6]:
from pyspark.sql.functions import col
df_employee.filter(col("name") == "aaa").show()
df_employee.filter(col("name") == "aaa").explain()

+---+----+
| id|name|
+---+----+
|100| aaa|
|200| aaa|
|220| aaa|
+---+----+

== Physical Plan ==
*(1) Filter (isnotnull(name#1) && (name#1 = aaa))
+- Scan ExistingRDD[id#0L,name#1]


In [7]:
df_employee.filter(col("name") == "aaa").explain(True)

== Parsed Logical Plan ==
'Filter ('name = aaa)
+- LogicalRDD [id#0L, name#1], false

== Analyzed Logical Plan ==
id: bigint, name: string
Filter (name#1 = aaa)
+- LogicalRDD [id#0L, name#1], false

== Optimized Logical Plan ==
Filter (isnotnull(name#1) && (name#1 = aaa))
+- LogicalRDD [id#0L, name#1], false

== Physical Plan ==
*(1) Filter (isnotnull(name#1) && (name#1 = aaa))
+- Scan ExistingRDD[id#0L,name#1]


In [8]:
df_employee.createOrReplaceTempView("vw_employee")

In [9]:
spark.sql("select * from vw_employee where name = 'aaa'").show()

+---+----+
| id|name|
+---+----+
|100| aaa|
|200| aaa|
|220| aaa|
+---+----+



In [10]:
spark.sql("select * from vw_employee where name = 'aaa'").explain(True)

== Parsed Logical Plan ==
'Project [*]
+- 'Filter ('name = aaa)
   +- 'UnresolvedRelation `vw_employee`

== Analyzed Logical Plan ==
id: bigint, name: string
Project [id#0L, name#1]
+- Filter (name#1 = aaa)
   +- SubqueryAlias `vw_employee`
      +- LogicalRDD [id#0L, name#1], false

== Optimized Logical Plan ==
Filter (isnotnull(name#1) && (name#1 = aaa))
+- LogicalRDD [id#0L, name#1], false

== Physical Plan ==
*(1) Filter (isnotnull(name#1) && (name#1 = aaa))
+- Scan ExistingRDD[id#0L,name#1]


In [13]:
tmp_list_dept = [(100,"finance"),(120,"marketing")]
top_row_dept = Row("id","dept_name")
tmp_rdd_dept = sc.parallelize(tmp_list_dept)
tmp_rdd_dept_schema = tmp_rdd_dept.map(lambda x: top_row_dept(*x)) 
df_dept = spark.createDataFrame(tmp_rdd_dept_schema)
df_dept.show()

+---+---------+
| id|dept_name|
+---+---------+
|100|  finance|
|120|marketing|
+---+---------+



In [8]:
df_employee.join(df_dept, df_dept.id == df_employee.id,"inner").show()

In [9]:
df_employee.join(df_dept, df_dept.id == df_employee.id,"inner").explain(True)

In [10]:
df_dept.createOrReplaceTempView("vw_dept")

In [11]:
spark.sql("select * from vw_employee join vw_dept on vw_employee.id = vw_dept.id ").show()

In [12]:
spark.sql("select * from vw_employee join vw_dept on vw_employee.id = vw_dept.id ").explain(True)

In [13]:
df_employee.join(df_dept.hint("broadcast"),"ID","left").show()

In [14]:
df_employee.join(df_dept,"id","left").explain()

In [15]:
df_employee.join(df_dept.hint("broadcast"),"ID","left").explain()

In [16]:
from pyspark.sql import Row
tmp_data = [(100,"aaa"),(120,"bbb"),(150,"ccc"),(200,"aaa"),(220,"aaa")]
tmp_row = Row("id","name")
rdd_row = sc.parallelize(tmp_data)
rdd_schema = rdd_row.map(lambda x: tmp_row(*x))

df_employee = spark.createDataFrame(rdd_schema)
df_employee.show()

In [17]:
#group by
df_employee.groupBy("name").count().sort("count").show()
df_employee.groupBy("name").count().sort("count").explain()

In [18]:
from pyspark.sql import Row
tmp_data = [(100,"aaa",5000),(120,"bbb",2000),(150,"ccc",5000),(200,"aaa",1000),(220,"aaa",3000)]
tmp_row = Row("id","name","salary")
rdd_row = sc.parallelize(tmp_data)
rdd_schema = rdd_row.map(lambda x: tmp_row(*x))

df_employee = spark.createDataFrame(rdd_schema)
df_employee.show()

In [19]:
df_employee.groupBy("name").sum("salary").show()
df_employee.groupBy("name").sum("salary").explain()

In [20]:
#Catalyst supports both rule-based and cost-based optimization.
1. cost based Optimizer(CBO): If a sql query can be executed in 2 different ways ( like may have path 1 and path2 for same query),then What CBO does is, it basically calculates the cost of each path and the analyses for which path the cost of execution is less and then executes that path so that it can optimize the quey execution.

2. Rule base optimizer(RBO): this basically follows the rules which are needed for executing a query. So depending on the number of rules which are to be applied, the optimzer runs the query.

In [21]:
df_employee.select("name","id").explain()
df_employee.select("name","id").limit(2).explain()